In [1]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import undetected_chromedriver as uc
from selenium_stealth import stealth
from selenium.webdriver.common.by import By

In [2]:
HOME_ADDRESS = "https://historicalnewspapers.lib.purdue.edu/?a=cl&cl=CL1&sp=PE&e=-------en-20--1--txt-txIN--------1"
options = Options()
#options.add_argument("--headless")
driver = uc.Chrome(options=options)
stealth(driver,
                languages=["en-US", "en"],
                vendor="Google Inc.",
                platform="Win32",
                webgl_vendor="Intel Inc.",
                renderer="Intel Iris OpenGL Engine",
                fix_hairline=True
        )

driver.implicitly_wait(5)
driver.get(HOME_ADDRESS)
input("Enter anything when captcha complete")
print(driver.title)

The Purdue Exponent (1889-2017) — Browse by title — Purdue University Newspapers Collection


In [3]:
start_year = 1936

In [4]:
calendar = driver.find_element(By.ID, 'datebrowserrichardtoplevelcalendar')

In [5]:
year = driver.find_element(By.XPATH, f"//h2[contains(text(), '{start_year}')]/following-sibling::div[1]")

In [6]:
first_link = year.find_element(
    By.XPATH,
    ".//ul[contains(@class,'nav') and contains(@class,'nav-pills')]//a[@class='nav-link'][1]"
)

In [7]:
first_link = first_link.get_attribute("href")

In [8]:
driver.get(first_link)

In [9]:
super_links = []

In [10]:
num_years = 14
num_months = 12 * num_years

for month in range(num_months + 1):
    try:
        calendar_box = driver.find_element(By.ID, "datebrowserrichardmonthlevelcalendar")
        weeks = calendar_box.find_elements(By.XPATH, "./*[position() > 1]")
        days = calendar_box.find_elements(
            By.XPATH,
            ".//div[@class='datebrowserrichardmonthlevelcalendardaycell' and not(contains(@class, ' '))]"
        )

        for day in days:
            date = day.find_element(By.CSS_SELECTOR, ".hiddenwhennotsmall")
            print(date.get_attribute("innerHTML"))
            link = day.find_element(By.CSS_SELECTOR, "li.list-group-item a")
            href = link.get_attribute("href")
            print(href)
            super_links.append((date.get_attribute("innerHTML"), href))
    except:
        pass
    next_month = driver.find_element(
        By.XPATH,
        "//a[span[text()='Next month']]"
    )
    href = next_month.get_attribute("href")
    print(href)
    driver.get(href)


Tuesday, 7 January 1936
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19360107-01&e=-------en-20--1--txt-txIN--------1
Wednesday, 8 January 1936
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19360108-01&e=-------en-20--1--txt-txIN--------1
Thursday, 9 January 1936
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19360109-01&e=-------en-20--1--txt-txIN--------1
Friday, 10 January 1936
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19360110-01&e=-------en-20--1--txt-txIN--------1
Saturday, 11 January 1936
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19360111-01&e=-------en-20--1--txt-txIN--------1
Tuesday, 14 January 1936
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19360114-01&e=-------en-20--1--txt-txIN--------1
Wednesday, 15 January 1936
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19360115-01&e=-------en-20--1--txt-txIN--------1
Friday, 17 January 1936
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19360117-01&e=-------en-20--1--t

In [11]:
import os

files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
files = [file.replace(".txt", "") for file in files]

In [12]:
def text_get():
    container = driver.find_element(By.ID, "documentdisplayleftpanesectiontextcontainer")

    paragraphs = container.find_elements(
        By.CSS_SELECTOR,
        "p[dir='auto']"
    )

    text = "\n".join([p.text for p in paragraphs])

    driver.get(super_link)

    return text

In [13]:
files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
files = [file.replace(".txt", "") for file in files]
while len(files) < len(super_links):
    for date, super_link in super_links:
        if date in files:
            continue
        driver.get(super_link)

        agg_text = ""
        container = driver.find_element(By.ID, "documentdisplayleftpanecontentscontainer")
        links = container.find_elements(By.CSS_SELECTOR, "a.pagetocnodelink")
        links = [link.get_attribute("href") for link in links]

        for link in links:
            driver.get(link)
            agg_text += text_get()
        if not agg_text:
            continue
        with open(f"/home/malinkyzubr/Desktop/HIST302/302hist/files/{date}.txt", "w") as f:
            f.write(agg_text)
    files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
    files = [file.replace(".txt", "") for file in files]

KeyboardInterrupt: 